In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import json
import torchvision
from torchvision import transforms
from torchvision.utils import save_image, make_grid
from torch.utils.data import Subset, random_split
from glob import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import pandas as pd
import shutil
from pycocotools import mask as coco_mask
import os
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
# import os

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!git clone https://github.com/jasonfghx/retina_image.git

fatal: destination path 'retina_image' already exists and is not an empty directory.


In [3]:
!pip3 install ultralytics

In [53]:
import cv2
import os
import numpy as np
os.makedirs('C:/Users/chenk/Desktop/attention/retina_image/train')
os.makedirs('C:/Users/chenk/Desktop/attention/retina_image/train/images')
os.makedirs('C:/Users/chenk/Desktop/attention/retina_image/train/labels')

FileExistsError: [WinError 183] 當檔案已存在時，無法建立該檔案。: 'C:/Users/chenk/Desktop/attention/retina_image/train'

In [54]:
add=glob("C:/Users/chenk/Desktop/attention/retina_image/Neo_Ridge/*")
for i in add[:]:
    image = cv2.imread(i)
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    enhanced_lab = cv2.merge([cl,a,b])
    image = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)
# 转换为灰度图像
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 应用高斯模糊以减少噪声
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    blurred=cv2.medianBlur(blurred,3)
# 使用自适应阈值分割图像
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 15, 10)

# 使用形态学操作以细化结果
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
# 使用自适应阈值分割图像
    rho = 1  # distance resolution in pixels of the Hough grid
    theta = np.pi / 180  # angular resolution in radians of the Hough grid
    threshold = 20  # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 110  # minimum number of pixels making up a line
    max_line_gap = 20  # maximum gap in pixels between connectable line segments

    lines = cv2.HoughLinesP(morph, rho, theta, threshold, np.array([]),
                    min_line_length, max_line_gap)
    point_p_l=[]
    for line in lines:
      for x1,y1,x2,y2 in line:
          if y1<1750 and y1>500 and x1<1750 and x1>500 and x2<1700:
            if image[x1,y1][0]<125 and image[x2,y2][0]<125 and image[x2,y2][1]<140 and image[x1,y1][1]<140 and image[x1,y1][2]<130:
                point_p_l.append((x1,y1))
                point_p_l.append((x2,y2))
    #             cv2.line(image,(x1,y1),(x2,y2),(255,0,0),5)
    # plt.imshow(image)
    # plt.show()
    if len(point_p_l)!=0:
        hull = cv2.convexHull(np.array(point_p_l))
        cv2.fillPoly(image, [np.array((hull))], (0, 0,0))
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# # 定義紅色範圍（根據需求調整）
    lower_red1 = np.array([0, 70, 50])  # 第一部分紅色範圍（低色調）
    upper_red1 = np.array([10, 255, 255])

    lower_red2 = np.array([170, 70, 50])  # 第二部分紅色範圍（高色調）
    upper_red2 = np.array([180, 255, 255])

# 創建紅色遮罩
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)

# 用黑色替換紅色部分
    image[red_mask > 0] = [0, 0, 0]
    #name=i.split("/")[-1]
    name = os.path.basename(i)

    cv2.imwrite(
    'C:/Users/chenk/Desktop/attention/retina_image/train/images/' + name,
    image
)

In [55]:
def mask_to_yolo_polygon(mask_image_path, path):
    label_file = open("C:/Users/chenk/Desktop/attention/retina_image/train/labels/" + str(path.replace('.png','')) + '.txt', 'a')
    mask = cv2.imread(mask_image_path, cv2.IMREAD_GRAYSCALE)
    h, w = mask.shape[0], mask.shape[1]
    _, binary = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #, 0.5, positive_orientation='low'

    for contour in contours:
        if len(contour) == 4 :continue
        # print(len(contour))
        label_file.write('0')
        label_file.write(' ')
        for cont in contour:
            label_file.write(str(cont[0][0] / w))
            label_file.write(' ')
            label_file.write(str(cont[0][1] / h))
            label_file.write(' ')
        label_file.write('\n')
    label_file.close()
for path in os.listdir("C:/Users/chenk/Desktop/attention/retina_image/Neo_Ridge_masks"):
    # print(path)
    mask_image_path = "C:/Users/chenk/Desktop/attention/retina_image/Neo_Ridge_masks/" + str(path)
    mask_to_yolo_polygon(mask_image_path, path)
#for path in os.listdir("/content/retina_image/s1/masks"):
    # print(path)
    #mask_image_path = "/content/retina_image/s1/masks/" + str(path)
    #mask_to_yolo_polygon(mask_image_path, path)

In [58]:
import yaml

d ={'train':"C:/Users/chenk/Desktop/attention/retina_image/train",'val': 'C:/Users/chenk/Desktop/attention/retina_image/train',
    'test':'C:/Users/chenk/Desktop/attention/retina_image/train','names':['d'],'nc':1}


with open('C:/Users/chenk/Desktop/attention/retina_image/data.yaml', 'w') as f:
    yaml.dump(d, f)

In [57]:
import torch, torchvision
from torchvision.ops import nms

print(torch.__version__)
print(torchvision.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)



2.7.1+cu118
0.22.1+cu118
True
11.8


In [59]:
from ultralytics import YOLO
model = YOLO("C:/Users/chenk/Desktop/attention/yolov8n-seg.pt")


# model =YOLO('/content/runs/segment/train2/weights/last.pt')

# Train the model
train_results = model.train(
    data="C:/Users/chenk/Desktop/attention/retina_image/data.yaml",  # path to dataset YAML
    epochs=1000,  # number of training epochs
    imgsz=640,batch=8,lr0=0.001, lrf=0.0005  # training image size
   
     # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

Ultralytics 8.3.252  Python-3.12.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/chenk/Desktop/attention/retina_image/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1000, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.0005, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:/Users/chenk/Desktop/attention/yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, 

In [60]:
results = model(source="C:/Users/chenk/Desktop/attention/retina_image/train/images/1.png", imgsz=640)
results  # 顯示結果
# results.save()


image 1/1 C:\Users\chenk\Desktop\attention\retina_image\train\images\1.png: 640x640 1 d, 24.1ms
Speed: 3.9ms preprocess, 24.1ms inference, 8.5ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'd'}
 obb: None
 orig_img: array([[[15, 15, 15],
         [15, 15, 15],
         [15, 15, 15],
         ...,
         [15, 15, 15],
         [15, 15, 15],
         [15, 15, 15]],
 
        [[15, 15, 15],
         [15, 15, 15],
         [15, 15, 15],
         ...,
         [15, 15, 15],
         [15, 15, 15],
         [15, 15, 15]],
 
        [[15, 15, 15],
         [15, 15, 15],
         [15, 15, 15],
         ...,
         [15, 15, 15],
         [15, 15, 15],
         [15, 15, 15]],
 
        ...,
 
        [[15, 15, 15],
         [15, 15, 15],
         [15, 15, 15],
         ...,
         [15, 15, 15],
         [15, 15, 15],
         [15, 15, 15]],
 
        [[15, 15, 15],
         [15, 15, 15],
         [15, 15, 15],
         ...,
         [15, 15, 15],
         [15, 15, 15],
         [15, 15, 15]],

In [61]:
from ultralytics import YOLO
model = YOLO("C:/Users/chenk/Desktop/attention/runs/segment/train/weights/last.pt")#改路徑


In [62]:
results = model(source="C:/Users/chenk/Desktop/attention/retina_image/train/images/1.png", imgsz=640)


image 1/1 C:\Users\chenk\Desktop\attention\retina_image\train\images\1.png: 640x640 1 d, 25.6ms
Speed: 3.3ms preprocess, 25.6ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)


In [70]:
# plt
plt.imshow(results[0].orig_img[:,:,::-1])
plt.show()
plt.imshow(results[0].masks.data.numpy()[0])
plt.show()

<Figure size 640x480 with 1 Axes>

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.